## Aca realizaré la descarga de las imagenes GOES

In [85]:
import GOES
import pyproj as pyproj
from pyresample import utils
from pyresample.geometry import SwathDefinition
from pyresample.kd_tree import resample_nearest

from goes2go.data import goes_latest, goes_nearesttime, goes_timerange
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cf
from glob import glob

In [62]:
#Download images with GOES package
files = GOES.download(Satellite='goes16', Product='ABI-L2-ACMF', DateTimeIni="20221120-180000", DateTimeFin="20221120-181000",
                      path_out='data/GOES/ABI-L2-ACMF/', rename_fmt='%Y%m%dT%H%M')

Files:
  OR_ABI-L2-ACMF-M6_G16_s20221120T1800.nc 100% 24.1MB 22s


In [84]:
#Open downloaded image and shrink domain the save image with lat/lon coordinates
paths = glob('data/GOES/ABI-L2-ACMF/*2022*.nc')
for f in paths:
    print(f)
    data_raw               = xr.open_dataset(f)
    x_attrs = data_raw.x.attrs
    y_attrs = data_raw.y.attrs

    x_attrs.pop('units')
    y_attrs.pop('units')

    data_attrs = data_raw.attrs
    HT_attrs   = data_raw.BCM.attrs
    # del data

    ds                 = GOES.open_dataset(f)
    domain             = [-90,-65,-50,-15]
    HT, LonCen, LatCen = ds.image('Cloud_Probabilities', lonlat='center', domain=domain)

    lon,lat = LonCen.data, LatCen.data
    # lon = np.diff(lon[1:,:])/2+lon[1:,1:]
    # lat = np.diff(lat[1:,:])/2+lat[1:,1:]
    lon_attrs = {'units':'degrees_east','long_name':'Longitude of center of pixels','standard_name':'pixel center longitude'}
    lat_attrs = {'units':'degrees_north','long_name':'Latitude of center of pixels','standard_name':'pixel center latitude'}

    data = xr.Dataset(coords={'y':np.arange(lat.shape[0]), 'x':np.arange(lon.shape[1])})
    data.coords['lat'] = (('y','x'), lat)
    data.coords['lon'] = (('y','x'), lon)
    data['HT'] = (('y','x'), HT.data)

    data.lon.attrs = lon_attrs
    data.lat.attrs = lat_attrs
    data.x.attrs = x_attrs
    data.y.attrs = y_attrs
    data.attrs = data_attrs
    data.coords['time'] = data_raw.t.values
    data = data.expand_dims('time')
    # data.to_netcdf('data/GOES/ABI-L2-ACHAF/processed/'+f.split("/")[-1])

data/GOES/ABI-L2-ACMF/OR_ABI-L2-ACMF-M6_G16_s20221120T1800.nc


AttributeError: NetCDF: Attribute not found